<a href="https://colab.research.google.com/github/kkaung66/CS498_LHW2_Codes/blob/main/Long_HW2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
from google.colab import files
import networkx as nx
import scipy.sparse as sp
import torch
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from math import exp
import sys

In [ ]:
uploaded = files.upload()

Saving sample_input.txt to sample_input (1).txt


In [ ]:
#reading data
def read_data(textfile):
    sample_input_import = open(textfile, 'r')
    given_inputs = []
    for ind, line in enumerate(sample_input_import):
        if ind == 0:
            total_edges = int(line)
        elif ind <= total_edges:
            user, ad, score = line.split(',')
            user, ad, score = int(user), int(ad), float(score)
            given_inputs.append([user, ad, score])
        else:
            query_user, query_ad = line.split(',')
            query_user, query_ad = int(query_user), int(query_ad)

    # assigning nodes and edges, 
    # this data sctructure was partially adapted from: https://github.com/taiyingchen/CS498HS/blob/master/HW3/simrank.py
    user_set = set()
    ad_set = set()
    user_links = {}
    ad_links = {}
    for users, ads, scores in given_inputs:
        if user not in user_set:
            user_links[users] = {}
        if ads not in ad_set:
            ad_links[ads] = {}
        user_set.add(users)
        ad_set.add(ads)
        user_links[users][ads] = scores
        ad_links[ads][users] = scores
    return user_set, ad_set, user_links, ad_links, query_user, query_ad



In [ ]:
## basic simrank implementation

## Initialization of simrank starts with either 1 if it is alpha(i,i) (same node), 
## or 0 if alpha(i,j) (different node)
def basic_simrank():
    k_interations = 10
    C1 = .8
    C2 = .8

    #initalization 
    user_simrank = {}
    for user_1 in user_set:
        user_simrank[user_1] = {}
        for user_2 in user_set:
            if user_1 == user_2:
                user_simrank[user_1][user_2] = 1
            else:
                user_simrank[user_1][user_2] = 0

    ad_simrank = {}
    for ad_1 in ad_set:
        ad_simrank[ad_1] = {}
        for ad_2 in ad_set:
            if ad_1 == ad_2:
                ad_simrank[ad_1][ad_2] = 1
            else:
                ad_simrank[ad_1][ad_2] = 0

    for k in range(k_interations):
        ## user simrank update
        for ind_1, user_1 in enumerate(user_set):
            for ind_2, user_2 in enumerate(user_set):
                if ind_1 <= ind_2:
                    sim_sum = 0
                    user_1_edges_total = len(user_links[user_1])
                    user_2_edges_total = len(user_links[user_2])
                    for user_1_ads in user_links[user_1]:
                        for user_2_ads in user_links[user_2]:
                            sim_sum += ad_simrank[user_1_ads][user_2_ads]
                    user_simrank[user_1][user_2] = (C1 * sim_sum) / (user_1_edges_total * user_2_edges_total)
                    user_simrank[user_2][user_1] = user_simrank[user_1][user_2]

        ## ad simrank update
        for ind_1, ad_1 in enumerate(ad_set):
            for ind_2, ad_2 in enumerate(ad_set):
                if ind_1 <= ind_2:
                    sim_sum = 0
                    ad_1_edges_total = len(ad_links[ad_1])
                    ad_2_edges_total = len(ad_links[ad_2])
                    for ad_1_users in ad_links[ad_1]:
                        for ad_2_users in ad_links[ad_2]:
                            sim_sum += user_simrank[ad_1_users][ad_2_users]
                    ad_simrank[ad_1][ad_2] = (C1 * sim_sum) / (ad_1_edges_total * ad_2_edges_total)
                    ad_simrank[ad_2][ad_1] = ad_simrank[ad_1][ad_2]
    return user_simrank, ad_simrank

In [ ]:
## basic simrank implementation with partial sum memoization

## Initialization of simrank starts with either 1 if it is alpha(i,i) (same node), 
## or 0 if alpha(i,j) (different node)
def partial_sum_sim():
    k_interations = 10
    C1 = .8
    C2 = .8
    user_simrank = {}
    for user_1 in user_set:
        user_simrank[user_1] = {}
        for user_2 in user_set:
            if user_1 == user_2:
                user_simrank[user_1][user_2] = 1
            else:
                user_simrank[user_1][user_2] = 0

    ad_simrank = {}
    for ad_1 in ad_set:
        ad_simrank[ad_1] = {}
        for ad_2 in ad_set:
            if ad_1 == ad_2:
                ad_simrank[ad_1][ad_2] = 1
            else:
                ad_simrank[ad_1][ad_2] = 0

    for k in range(k_interations):
        ## user simrank update
        partial_sum = {}
        for users in user_set:
            partial_sum[users] = {}
            for ads in ad_set:
                partial_sum[users][ads] = 0
                for edges in user_links[users]:
                    partial_sum[users][ads] += ad_simrank[ads][edges]

        for ind_1, user_1 in enumerate(user_set):
            for ind_2, user_2 in enumerate(user_set):
                if ind_1 <= ind_2:
                    sim_sum = 0
                    user_1_edges_total = len(user_links[user_1])
                    user_2_edges_total = len(user_links[user_2])
                    for user_1_ads in user_links[user_1]:
                        sim_sum += partial_sum[user_1][user_1_ads]
                    user_simrank[user_1][user_2] = (C1 * sim_sum) / (user_1_edges_total * user_2_edges_total)
                    user_simrank[user_2][user_1] = user_simrank[user_1][user_2]

        partial_sum = {}
        for ads in ad_set:
            partial_sum[ads] = {}
            for users in user_set:
                partial_sum[ads][users] = 0
                for edges in ad_links[ads]:
                    partial_sum[ads][users] += user_simrank[users][edges]
        ## ad simrank update
        for ind_1, ad_1 in enumerate(ad_set):
            for ind_2, ad_2 in enumerate(ad_set):
                if ind_1 <= ind_2:
                    sim_sum = 0
                    ad_1_edges_total = len(ad_links[ad_1])
                    ad_2_edges_total = len(ad_links[ad_2])
                    for ad_1_users in ad_links[ad_1]:
                        sim_sum += partial_sum[ad_1][ad_1_users]
                    ad_simrank[ad_1][ad_2] = (C1 * sim_sum) / (ad_1_edges_total * ad_2_edges_total)
                    ad_simrank[ad_2][ad_1] = ad_simrank[ad_1][ad_2]
    return user_simrank, ad_simrank

In [ ]:
# Edvidence geometric and exponential
def edv_geo_exp(simrank, links, usr_ad_set):
    geo_simrank = simrank.copy()
    exp_simrank = simrank.copy()
    user_evidence_geometric_a_b = {}
    user_evidence_exponential_a_b = {}
    user_geo_simrank = {}
    user_exp_simrank = {}
    for user_a in usr_ad_set:
        user_a_edges = set(links[user_a])
        user_evidence_geometric_a_b[user_a] = {}
        user_evidence_exponential_a_b[user_a] = {}
        for user_b in usr_ad_set:
            user_b_edges = set(links[user_b])
            numbers_of_edges_intersection = len(user_a_edges.intersection(user_b_edges))

            # exponential simrank calculation
            user_evidence_exponential_a_b[user_a][user_b] = 1 - np.exp(-1*numbers_of_edges_intersection)
            exp_simrank[user_a][user_b] *= user_evidence_exponential_a_b[user_a][user_b]
            exp_simrank[user_b][user_a] = geo_simrank[user_a][user_b]

            # geometric simrank calculation
            user_evidence_geometric_a_b[user_a][user_b] = 0
            for i in range(1,numbers_of_edges_intersection+1):
                user_evidence_geometric_a_b[user_a][user_b] += 1 / (2 ** i)
            geo_simrank[user_a][user_b] *= user_evidence_geometric_a_b[user_a][user_b]
            geo_simrank[user_b][user_a] = geo_simrank[user_a][user_b]

    return exp_simrank, geo_simrank

In [ ]:
def output_sorted_ad_simrank(ad_simrank):
    query_ad_similar_ads = []
    for ads in ad_simrank[query_ad]:
        query_ad_similar_ads.append(ad_simrank[query_ad][ads])
    query_ad_similar_ads = sorted(query_ad_similar_ads, reverse = True)
    sorted_similar_ads = []
    for ranks in query_ad_similar_ads:
        for ads in sorted(ad_simrank[query_ad]):
            if ranks == ad_simrank[query_ad][ads]:
                sorted_similar_ads.append(ads)
    print(sorted_similar_ads[1], sorted_similar_ads[2], sorted_similar_ads[3])
    #print("Most similar ads to Qa using simple_simrank:", sorted_similar_ads[1], sorted_similar_ads[2], sorted_similar_ads[3])
    return None

def output_sorted_user_simrank(user_simrank):
    query_user_similar_users = []
    for users in user_simrank[query_user]:
        query_user_similar_users.append(user_simrank[query_user][users])
    query_user_similar_users = sorted(query_user_similar_users, reverse = True)
    sorted_similar_users = []
    for ranks in query_user_similar_users:
        for users in sorted(user_simrank[query_user]):
            if ranks == user_simrank[query_user][users]:
                sorted_similar_users.append(users)
    print(sorted_similar_users[1], sorted_similar_users[2], sorted_similar_users[3])
    #print("Most similar users to Qu using simple_simrank:", sorted_similar_users[1], sorted_similar_users[2], sorted_similar_users[3])

    return None

In [ ]:
uploaded = files.upload()

Saving input_b.txt to input_b.txt


In [ ]:
user_set, ad_set, user_links, ad_links, query_user, query_ad = read_data("input_b.txt")
user_simrank, ad_simrank = basic_simrank()
user_simrank_partial , ad_simrank_partial = partial_sum_sim()
user_simrank_partial_exp, user_simrank_partial_geo = edv_geo_exp(user_simrank_partial, user_links, user_set)
ad_simrank_partial_exp, ad_simrank_partial_geo = edv_geo_exp(ad_simrank_partial, ad_links, ad_set)

output_sorted_user_simrank(user_simrank_partial)
output_sorted_ad_simrank(ad_simrank_partial)
output_sorted_user_simrank(user_simrank_partial)
output_sorted_ad_simrank(ad_simrank_partial_exp)
output_sorted_user_simrank(user_simrank_partial)
output_sorted_ad_simrank(ad_simrank_partial_geo)


1 5 1
76584 579 661
1 5 1
76584 579 661
1 5 1
76584 579 661
